In [73]:
import numpy as np
import scipy.special
from scipy.integrate import trapezoid
from numpy import cos, sin
from numpy.linalg import norm

In [74]:
def mathieu_cem(m, q, x):
    return scipy.special.mathieu_cem(m, q, 180/np.pi*x)
def mathieu_sem(m, q, x):
    return scipy.special.mathieu_sem(m, q, 180/np.pi*x)

In [75]:
def ce_q_expansion(m,q,v):
    # This implements the general expansion for small q shown
    # in DLMF 28.6.26

    t1 = cos(m*v)
    t2 = q*(cos((m+2)*v)/(m+1) - cos((m-2)*v)/(m-1))/4
    t3 = (q**2)*(cos((m+4)*v)/((m+1)*(m+2)) + cos((m-4)*v)/((m-1)*(m-2)) - 2*(m*m+1)/((m*m-1)**2)*cos(m*v))/32
    ce = t1-t2+t3
    return ce

In [76]:
fail = 0
    
qs = np.logspace(-3,3,21)
qs = np.hstack([-qs, qs])

N = 1000
v = np.linspace(-np.pi,np.pi, N)

In [77]:
MM = 10

#====================================================
# First test normalization per DLMF 28.2.30

tol = 1e-13

for m in range(0, MM+1):
    for i in range(len(qs)):
        q = qs[i]
    
        ce, _ = mathieu_cem(m,q,v)
        s = trapezoid(ce*ce,v)
    
        diff = s - np.pi
    
        if (abs(diff) > tol):
        
            fail = fail+1  


In [78]:
from itertools import combinations
from scipy.integrate import quad
#====================================================
# Next test orthogonality per DLMF 28.2,31

tol = 1e-10
MM = 5  # Max order to test
for m1, m2 in combinations(range(0,MM+1),2):

    print(f'-----------  [m1,m2] = [{m1},{m2}]  -----------\n')
    for i in range(len(qs)):
        q = qs[i]
        
        #ce1 = mathieu_ce(m1,q,v);
        #ce2 = mathieu_ce(m2,q,v);     
        #s = trapz(v,ce1.*ce2);

        # integral is recommended over trapz
        # For some reason, I need to transpose the output of mathieu_ce.      
        ce = lambda m,q,v: mathieu_cem(m,q,v)[0] 
        f = lambda v: ce(m1,q,v)*ce(m2,q,v)
        s = quad(f, -np.pi, np.pi)[0]
        
        diff = s
        print(f's = {s}, err = {diff}\n')
        if (abs(diff) > tol):
            print(f'Error!  [m1,m2] = [{m1},{m2}], q = {q}, diff = {diff}\n')
            fail = fail+1

    


-----------  [m1,m2] = [0,1]  -----------

s = -7.411720558268343e-16, err = -7.411720558268343e-16

s = -8.839648981991021e-16, err = -8.839648981991021e-16

s = -8.561817485948777e-16, err = -8.561817485948777e-16

s = -8.496694685681179e-16, err = -8.496694685681179e-16

s = -8.574975695697224e-16, err = -8.574975695697224e-16

s = -8.935786526912907e-16, err = -8.935786526912907e-16

s = -9.056456797293484e-16, err = -9.056456797293484e-16

s = -9.733598583917315e-16, err = -9.733598583917315e-16

s = -9.651477678224707e-16, err = -9.651477678224707e-16

s = -1.1822882345367121e-15, err = -1.1822882345367121e-15

s = -1.3768514627759765e-15, err = -1.3768514627759765e-15

s = -1.6279643570463073e-15, err = -1.6279643570463073e-15

s = -1.913103924141947e-15, err = -1.913103924141947e-15

s = -2.347900135459166e-15, err = -2.347900135459166e-15

s = -2.3776287705501695e-15, err = -2.3776287705501695e-15

s = -2.7212439629314376e-15, err = -2.7212439629314376e-15

s = -2.767663853177

In [79]:
#====================================================
# Test q = 0 case per DLMF 28.2.29
print('Test ce tends to cos for q = -1e-13 per DLMF 28.2.29 ... \n')
tol = 5e-13
q = -1e-13
MM = 10;  # Max order to test
for m in range(0,MM+1):
    print(f'-----------  m = {m}  -----------\n')
    LHS, _ = mathieu_cem(m,q,v)
    RHS = np.cos(m*v)

    ndiff = np.linalg.norm(LHS-RHS)
    print(f'm = {m}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, norm err = {ndiff}\n')
    if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, ndiff = {ndiff}\n')
        fail = fail+1
    

Test ce tends to cos for q = -1e-13 per DLMF 28.2.29 ... 

-----------  m = 0  -----------

m = 0, LHS(1) = 0.7071067811865828, RHS(1) = 1.0, norm err = 9.262096826685895

Error!  m = 0, q = -1e-13, LHS(1) = 0.7071067811865828, RHS(1) = 1.0, ndiff = 9.262096826685895

-----------  m = 1  -----------

m = 1, LHS(1) = -0.9999802213186958, RHS(1) = -0.9999802213186832, norm err = 2.7863592378044743e-13

-----------  m = 2  -----------

m = 2, LHS(1) = 0.9999208860571005, RHS(1) = 0.9999208860571255, norm err = 8.675021777799014e-13

Error!  m = 2, q = -1e-13, LHS(1) = 0.9999208860571005, RHS(1) = 0.9999208860571255, ndiff = 8.675021777799014e-13

-----------  m = 3  -----------

m = 3, LHS(1) = -0.9998219965624606, RHS(1) = -0.9998219965624732, norm err = 3.230696003729418e-13

-----------  m = 4  -----------

m = 4, LHS(1) = 0.9996835567465256, RHS(1) = 0.9996835567465338, norm err = 2.15750999566844e-13

-----------  m = 5  -----------

m = 5, LHS(1) = -0.9995055720856155, RHS(1) = -0.9

In [80]:
print('Test ce tends to cos for q = 0 per DLMF 28.2.29 ... \n')
q = 0
MM = 10;  # Max order to test
for m in range(0,MM+1):
    print(f'-----------  m = {m}  -----------\n')
    LHS, _ = mathieu_cem(m,q,v)
    RHS = np.cos(m*v)

    ndiff = np.linalg.norm(LHS-RHS)
    print(f'm = {m}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, norm err = {ndiff}\n')
    if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, ndiff = {ndiff}\n')
        fail = fail+1


Test ce tends to cos for q = 0 per DLMF 28.2.29 ... 

-----------  m = 0  -----------

m = 0, LHS(1) = 0.7071067811865475, RHS(1) = 1.0, norm err = 9.262096826685903

Error!  m = 0, q = 0, LHS(1) = 0.7071067811865475, RHS(1) = 1.0, ndiff = 9.262096826685903

-----------  m = 1  -----------

m = 1, LHS(1) = -0.9999802213186832, RHS(1) = -0.9999802213186832, norm err = 8.462233603574857e-15

-----------  m = 2  -----------

m = 2, LHS(1) = 0.9999208860571255, RHS(1) = 0.9999208860571255, norm err = 1.823824376709566e-14

-----------  m = 3  -----------

m = 3, LHS(1) = -0.9998219965624732, RHS(1) = -0.9998219965624732, norm err = 2.958805648943347e-14

-----------  m = 4  -----------

m = 4, LHS(1) = 0.9996835567465339, RHS(1) = 0.9996835567465338, norm err = 3.684447532658608e-14

-----------  m = 5  -----------

m = 5, LHS(1) = -0.9995055720856216, RHS(1) = -0.9995055720856215, norm err = 4.757961095915622e-14

-----------  m = 6  -----------

m = 6, LHS(1) = 0.9992880496203402, RHS(1)

In [81]:
print('Test ce tends to cos for q = 1e-13 per DLMF 28.2.29 ... \n')
q = 1e-13
MM = 10  # Max order to test
for m in range(0,MM+1):
    print('-----------  m = %d  -----------\n', m)
    LHS, _ = mathieu_cem(m,q,v)
    RHS = np.cos(m*v)

    ndiff = np.linalg.norm(LHS-RHS);
    print(f'm = {m}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, norm err = {ndiff}\n')
    if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, LHS(1) = {LHS[1]}, RHS(1) = {RHS[1]}, ndiff = {ndiff}\n')
        fail = fail+1

Test ce tends to cos for q = 1e-13 per DLMF 28.2.29 ... 

-----------  m = %d  -----------
 0
m = 0, LHS(1) = 0.7071067811865122, RHS(1) = 1.0, norm err = 9.262096826685898

Error!  m = 0, q = 1e-13, LHS(1) = 0.7071067811865122, RHS(1) = 1.0, ndiff = 9.262096826685898

-----------  m = %d  -----------
 1
m = 1, LHS(1) = -0.9999802213186707, RHS(1) = -0.9999802213186832, norm err = 2.809173235069021e-13

-----------  m = %d  -----------
 2
m = 2, LHS(1) = 0.9999208860571505, RHS(1) = 0.9999208860571255, norm err = 8.81457924873526e-13

Error!  m = 2, q = 1e-13, LHS(1) = 0.9999208860571505, RHS(1) = 0.9999208860571255, ndiff = 8.81457924873526e-13

-----------  m = %d  -----------
 3
m = 3, LHS(1) = -0.9998219965624857, RHS(1) = -0.9998219965624732, norm err = 3.4507913822873964e-13

-----------  m = %d  -----------
 4
m = 4, LHS(1) = 0.9996835567465422, RHS(1) = 0.9996835567465338, norm err = 2.4053867058721625e-13

-----------  m = %d  -----------
 5
m = 5, LHS(1) = -0.9995055720856278

In [82]:
# ====================================================
# Next test even fcns per DLMF 28.2,34
print('Test rotation identity for even fcns per DLMF 28.2,34 ... \n')
tol = 1e-13
v = 0.05  # Just check one random point.
MM = 10  # Sets max order to test
for m in range(0,MM,2):
    ss = ((-1)**((m)/2))
    print(f'-----------  m = {m}, ss = {ss}  -----------\n')
    print(f's = {s}\n')
    for i in range(len(qs)):
        q = qs[i]
        LHS = mathieu_cem(m,-q,v)[0]
        RHS = ss*mathieu_cem(m,q,(np.pi/2)-v)[0]
        diff = LHS-RHS
        print(f'm = {m}, LHS = {LHS}, RHS = {RHS}, err = {diff}\n')
        if (abs(diff) > tol):
            print(f'Error!  m = {m}, q = {q}, LHS = {LHS}, RHS = {RHS}, diff = {diff}\n', m, q, LHS, RHS, diff)
            fail = fail+1
    

Test rotation identity for even fcns per DLMF 28.2,34 ... 

-----------  m = 0, ss = 1.0  -----------

s = -7.529635199885235e-15

m = 0, LHS = 0.7067549716125878, RHS = 0.7067549716125878, err = 0.0

m = 0, LHS = 0.7064047844025648, RHS = 0.7064047844025648, err = 0.0

m = 0, LHS = 0.7057059381152234, RHS = 0.7057059381152234, err = 0.0

m = 0, LHS = 0.7043110449768635, RHS = 0.7043110449768635, err = 0.0

m = 0, LHS = 0.7015259092096182, RHS = 0.7015259092096182, err = 0.0

m = 0, LHS = 0.6959616610646315, RHS = 0.6959616610646315, err = 0.0

m = 0, LHS = 0.6848359618134272, RHS = 0.6848359618134272, err = 0.0

m = 0, LHS = 0.662583438408093, RHS = 0.662583438408093, err = 0.0

m = 0, LHS = 0.6182873697018711, RHS = 0.6182873697018711, err = 0.0

m = 0, LHS = 0.5326851865242502, RHS = 0.5326851865242502, err = 0.0

m = 0, LHS = 0.3860086379642601, RHS = 0.3860086379642601, err = 0.0

m = 0, LHS = 0.2045982935602517, RHS = 0.2045982935602517, err = 0.0

m = 0, LHS = 0.0716036541976564

In [83]:
#====================================================
#Next test odd fcns per DLMF 28.2,35
print('Test rotation identity for odd fcns per DLMF 28.2,35 ... \n')
v = 0.05;  # Just check one random point.
#MM = 10;  # Sets max order to test
for m in range(0, MM+1, 2):
  ss = ((-1)**((m-1)/2))
  print(f'-----------  m = {m}, ss = {ss}  -----------\n')
  #print('s = %f\n', s)
  for i in range(len(qs)):
    q = qs[i]
    LHS = mathieu_cem(m,-q,v)[0]
    RHS = ss*mathieu_sem(m,q,np.pi/2-v)[0]
    diff = LHS-RHS
    print(f'm = {m}, LHS = {LHS}, RHS = {RHS}, err = {diff}\n')
    if (abs(diff) > tol):
      print(f'Error!  m = {m}, q = {q}, LHS = {LHS}, RHS = {RHS}, diff = {diff}\n')
      fail = fail+1


Test rotation identity for odd fcns per DLMF 28.2,35 ... 

-----------  m = 0, ss = (6.123233995736766e-17-1j)  -----------

m = 0, LHS = 0.7067549716125878, RHS = 0j, err = (0.7067549716125878+0j)

Error!  m = 0, q = -0.001, LHS = 0.7067549716125878, RHS = 0j, diff = (0.7067549716125878+0j)

m = 0, LHS = 0.7064047844025648, RHS = 0j, err = (0.7064047844025648+0j)

Error!  m = 0, q = -0.001995262314968879, LHS = 0.7064047844025648, RHS = 0j, diff = (0.7064047844025648+0j)

m = 0, LHS = 0.7057059381152234, RHS = 0j, err = (0.7057059381152234+0j)

Error!  m = 0, q = -0.003981071705534973, LHS = 0.7057059381152234, RHS = 0j, diff = (0.7057059381152234+0j)

m = 0, LHS = 0.7043110449768635, RHS = 0j, err = (0.7043110449768635+0j)

Error!  m = 0, q = -0.007943282347242814, LHS = 0.7043110449768635, RHS = 0j, diff = (0.7043110449768635+0j)

m = 0, LHS = 0.7015259092096182, RHS = 0j, err = (0.7015259092096182+0j)

Error!  m = 0, q = -0.015848931924611134, LHS = 0.7015259092096182, RHS = 0j, di

In [84]:
#====================================================
# Next test small q expansions per DLMF 28.6.21.  The goal
# is to make sure my fcn impls match the DLMF for negative
# q values.
print('Test small q expansions per DLMF 28.6.21 ... \n')

tol = 1e-4;  #High tol since these expansions have small ROC.
N = 1000
v = np.linspace(-np.pi,np.pi,N).reshape((-1,1));  # Col vector.

MM = 50

# The first three orders don't work unless q is really small
# -----------------------------------------------------------
# m = 0
qs = [-.2, -.1, .1, .2];  
for i in range(len(qs)):
    q = qs[i]
    # Define power series after setting q.
    ce0 = lambda v: (1/np.sqrt(2))*(1 - q*np.cos(2*v)/2 + (q**2)*(np.cos(4*v-2))/32 - (q**3)*(np.cos(6*v)/9 - 11*np.cos(2*v))/128)
    m = 0
    print(f'----------- m = {m}, q = {q}  -----------\n', m, q)  
    dlmfce0 = ce0(v)
    myce0 = mathieu_cem(m,q,v)[0]
    #plot(v, dlmfce0)
    #hold on
    #plot(v, myce0)
    #legend('dlmf','me')
    ndiff = np.linalg.norm(dlmfce0 - myce0)/N
    if (ndiff > tol):
        print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
        fail = fail+1



Test small q expansions per DLMF 28.6.21 ... 

----------- m = 0, q = -0.2  -----------
 0 -0.2
----------- m = 0, q = -0.1  -----------
 0 -0.1
----------- m = 0, q = 0.1  -----------
 0 0.1
----------- m = 0, q = 0.2  -----------
 0 0.2


In [85]:
#Expansion test contd 1
#-----------------------------------------------------------  
#m = 1
qs = [-1.5, -1, -.5, -.2, -.1, .1, .2, .5, 1, 1.5];    
for i in range(len(qs)):
    q = qs[i]
    # Define power series after setting q.
    ce1 = lambda v: cos(v) - q*cos(3*v)/8 + (q**2)*(2*cos(5*v)/3 - 2*cos(3*v) - cos(v))/128 - (q**3)*(cos(7*v)/9 - 8*cos(5*v)/9 - cos(3*v)/3 + 2*cos(v))/1024
    m = 1
    print(f'----------- m = {m}, q = {q}  -----------\n')  
    dlmfce1 = ce1(v)
    myce1 = mathieu_cem(m,q,v)[0]
    ndiff = norm(dlmfce1 - myce1)/N
    if (ndiff > tol):
      print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n', m, q, ndiff)
      fail = fail+1


----------- m = 1, q = -1.5  -----------

----------- m = 1, q = -1  -----------

----------- m = 1, q = -0.5  -----------

----------- m = 1, q = -0.2  -----------

----------- m = 1, q = -0.1  -----------

----------- m = 1, q = 0.1  -----------

----------- m = 1, q = 0.2  -----------

----------- m = 1, q = 0.5  -----------

----------- m = 1, q = 1  -----------

----------- m = 1, q = 1.5  -----------



In [86]:
#-----------------------------------------------------------  
m = 2
qs = [-.5, -.2, -.1, .1, .2, .5]
print(f'{tol=}')
for i in range(len(qs)):
    q = qs[i]
    # Define power series after setting q.
    ce2 = lambda v: cos(2*v) - q*(cos(4*v)/3 - 1)/4 + (q**2)*(cos(6*v)/3 - 76*cos(2*v)/9)/128
    print(f'----------- m = {m}, q = {q}  -----------\n')  
    dlmfce2 = ce2(v)
    myce2 = mathieu_cem(m,q,v)[0]
    ndiff = norm(dlmfce2 - myce2)/N
    if (ndiff > tol):
      print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
      fail = fail+1


tol=0.0001
----------- m = 2, q = -0.5  -----------

Error!  m = 2, q = -0.5, ndiff = 0.00015866310545473652

----------- m = 2, q = -0.2  -----------

----------- m = 2, q = -0.1  -----------

----------- m = 2, q = 0.1  -----------

----------- m = 2, q = 0.2  -----------

----------- m = 2, q = 0.5  -----------

Error!  m = 2, q = 0.5, ndiff = 0.00015859781866647206



In [87]:
#-----------------------------------------------------------  
# Higher orders work over larger q domains
# Now m = 3, 4, 5, ... 11
qs = [-1.5, -1, -.5, -.1, .1, .5, 1, 1.5];  
for i in range(len(qs)):
  q = qs[i]
  for m in range(3,11+1):
    print(f'----------- m = {m}, q = {q} -----------\n')  
    dlmfce = ce_q_expansion(m,q,v)
    myce = mathieu_cem(m,q,v)[0]
    ndiff = norm(dlmfce - myce)/N
    if (ndiff > tol):
      print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
      fail = fail+1


----------- m = 3, q = -1.5 -----------

Error!  m = 3, q = -1.5, ndiff = 0.0001318524774592979

----------- m = 4, q = -1.5 -----------

----------- m = 5, q = -1.5 -----------

----------- m = 6, q = -1.5 -----------

----------- m = 7, q = -1.5 -----------

----------- m = 8, q = -1.5 -----------

----------- m = 9, q = -1.5 -----------

----------- m = 10, q = -1.5 -----------

----------- m = 11, q = -1.5 -----------

----------- m = 3, q = -1 -----------

----------- m = 4, q = -1 -----------

----------- m = 5, q = -1 -----------

----------- m = 6, q = -1 -----------

----------- m = 7, q = -1 -----------

----------- m = 8, q = -1 -----------

----------- m = 9, q = -1 -----------

----------- m = 10, q = -1 -----------

----------- m = 11, q = -1 -----------

----------- m = 3, q = -0.5 -----------

----------- m = 4, q = -0.5 -----------

----------- m = 5, q = -0.5 -----------

----------- m = 6, q = -0.5 -----------

----------- m = 7, q = -0.5 -----------

----------- m =

In [88]:
#-----------------------------------------------------------  
# Now even higher orders
qs = [-10, -3, -1.5, -1, -.5, -.1, .1, .5, 1, 1.5, 3, 10];  
for i in range(len(qs)):
    q = qs[i]
    for m in range(12, MM+1):
        print(f'----------- m = {m}, q = {q}  -----------\n')  
        dlmfce = ce_q_expansion(m,q,v)
        myce = mathieu_cem(m,q,v)[0]
        ndiff = norm(dlmfce - myce)/N
        if (ndiff > tol):
            print(f'Error!  m = {m}, q = {q}, ndiff = {ndiff}\n')
            fail = fail+1



----------- m = 12, q = -10  -----------

Error!  m = 12, q = -10, ndiff = 0.00016180245253813778

----------- m = 13, q = -10  -----------

Error!  m = 13, q = -10, ndiff = 0.00012590163813512706

----------- m = 14, q = -10  -----------

----------- m = 15, q = -10  -----------

----------- m = 16, q = -10  -----------

----------- m = 17, q = -10  -----------

----------- m = 18, q = -10  -----------

----------- m = 19, q = -10  -----------

----------- m = 20, q = -10  -----------

----------- m = 21, q = -10  -----------

----------- m = 22, q = -10  -----------

----------- m = 23, q = -10  -----------

----------- m = 24, q = -10  -----------

----------- m = 25, q = -10  -----------

----------- m = 26, q = -10  -----------

----------- m = 27, q = -10  -----------

----------- m = 28, q = -10  -----------

----------- m = 29, q = -10  -----------

----------- m = 30, q = -10  -----------

----------- m = 31, q = -10  -----------

----------- m = 32, q = -10  -----------

----

In [89]:
print(f'At end, fail = {fail}\n')

At end, fail = 260

